In [ ]:
import os
import astropy.io.fits as fits
import numpy as np
import scipy
import scipy.ndimage as ndi
import matplotlib.pylab as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pyklip.klip
import pyklip.instruments.Instrument as Instrument
import pyklip.parallelized as parallelized
import pyklip.rdi as rdi
import pyklip.fakes as fakes
import glob
from astropy.table import Table
from astropy.table import join
from astropy.table import vstack
import pandas as pd
import pdb

# Creating Contrast Curves

Using NIRCAM data simulated by the Space Telescope Science Institute, we'd like to estimate the contrast needed by a given planet at different separations from its host star in order for it to be detectable by the James Webb Space Telescope. This notebook performs an analysis of contrast in starlight-subtracted images in order to generate contrast curves for the visualization of this information.
The first step in this process is thus utilzing the pyKLIP algorithm to remove starlight, then measuring the contrast in the image at varying distances. However, this PSF subtraction can result in the distortion of planet spectrum, and an oversubtraction of flux from the image as a whole. Thereofore, we also measure the algortithm throughput as a function of planet sepration.

### Loading the dataset

This loads loads in our simulated JWST data, specifying the roll angle (10 arcseconds is the maximum roll angle) as well as the center as the image and the inner working angle. It combines images from both roll angles (0 and 10 arcseconds) to create a full sequence of images.

In [ ]:
#Import the dataset to be used
filtername = "f300m"

# read in roll 1
with fits.open("old_simulated_data/NIRCam_target_Roll1_{0}.fits".format(filtername)) as hdulist:
    roll1_cube = hdulist[0].data

# read in roll 2
with fits.open("old_simulated_data/NIRCam_target_Roll2_{0}.fits".format(filtername)) as hdulist:
    roll2_cube = hdulist[0].data  

# combine the two rows
full_seq = np.concatenate([roll1_cube, roll2_cube], axis=0)

# two rolls are offset 10 degrees
pas = np.append([0 for _ in range(roll1_cube.shape[0])], [10 for _ in range(roll2_cube.shape[0])])

# for each image, the (x,y) center where the star is is just the center of the image
centers = np.array([np.array(frame.shape)/2. for frame in full_seq])

# give it some names, just in case we want to refer to them
filenames = np.append(["roll1_{0}".format(i) for i in range(roll1_cube.shape[0])],
                      ["roll2_{0}".format(i) for i in range(roll1_cube.shape[0])])

#Define dataset
dataset = Instrument.GenericData(full_seq, centers, IWA=4, parangs=pas, filenames=filenames)
dataset.flipx = False

### Run Klip

Use the pyklip algortihm on the dataset in order to remove starlight from the images. Using KLIP-ADI reductions, we will break the image into 9 concentric annuli, and each annuli into 4 azimuthal sectors.

In [ ]:
parallelized.klip_dataset(dataset, outputdir="./", fileprefix="pyklip-f300m-ADI-k50a9s4m1", annuli=9, 
                          subsections=4, numbasis=[1,5,10,20,50], mode="ADI", movement=1)

### Mask Any Real Planets in Image

There are two "real" planets already in this dataset, but we don't want to include them in our noise estimation. Their positions are at (41,54) and (43,70) respectively. We will therefore mask these this positions with nans before calculating the contrast in our images. We used 5 KL modes to KLIP our data, but to create the contrast curve we only need 1 frame. We'll arbitrarily chose the KL mode with an index of 2 to accomplish this. 

In [ ]:
#Read in the KLIP-ed dataset
with fits.open("pyklip-f300m-ADI-k50a9s4m1-KLmodes-all.fits") as hdulist:
    adi_cube = hdulist[0].data
    adi_centers = [hdulist[0].header['PSFCENTX'], hdulist[0].header['PSFCENTY']]

    
#Plot the KL10 Cube (index of 2)
plt.figure() 
plt.imshow(adi_cube[2], interpolation='nearest', cmap='inferno')
plt.gca().invert_yaxis()

In [ ]:
#Mask the 1st planet
planet1_peak_x = 41 
planet1_peak_y = 54

#Create an array with the indices are that of KL mode frame with index 2
ydat, xdat = np.indices(adi_cube[2].shape)

#Set the FWHM of the PSF
psf_fwhm = 6

#Calculate the distance around the planet to be masked
distance_from_planet1 = np.sqrt((xdat - planet1_peak_x)**2 + (ydat - planet1_peak_y)**2)

#Mask
adi_cube[2][np.where(distance_from_planet1 <= 2*psf_fwhm)] = np.nan

#fakes.gaussfit2d(psf_frame, 71, 30, searchrad=3, guessfwhm=2, guesspeak=1, refinefit=True)
#Use pyklip.fakes to gaussfit2d and centroid on tiny planets
#Mask the second planet
planet2_peak_x = 43
planet2_peak_y = 70
distance_from_planet2 = np.sqrt((xdat - planet2_peak_x)**2 + (ydat - planet2_peak_y)**2)
adi_cube[2][np.where(distance_from_planet2 <= 2*psf_fwhm)] = np.nan

#plot the new masked data
plt.figure()
plt.imshow(adi_cube[2], interpolation='nearest', cmap='inferno')
plt.gca().invert_yaxis()

### Measure the Contrast

Using the pyKLIP function meas_contrast, we can compute the 5 $\sigma$ noise at each separation in our image. For this function, we again need to specify our planet's FWHM as well as our outer working angle and the center of our input frame. 

In [ ]:
OWA = 65 #Setting this to be distance between star and outer edge of image

#Measuring the contrast in the image
contrast_seps, contrast = pyklip.klip.meas_contrast(dat = adi_cube[2], 
                                                     iwa = dataset.IWA, 
                                                     owa = OWA, 
                                                     resolution = (psf_fwhm), 
                                                     center = adi_centers,
                                                     low_pass_filter = False)

In [ ]:
#Plot contrast curve!
plt.figure(figsize = (15,10))
plt.plot(contrast_seps, contrast, color = "teal")
plt.xlabel("Separation (pixels)")
plt.ylabel("Contrast (5 $\sigma$)")
plt.title("Non-Calibrated Contrast Curve")
plt.rcParams.update({'font.size': 22})
plt.savefig("contrast_adi2.png", dpic = 500)

## Computing throughput

Now that we've created our contrast curve, we can calculate the throughput of our KLIP reduced images. In order to optimize this calculation, we need to inject multiple fake planets at varying separations and postion angles to get a feel for how throughput changes across the image.

### The Fake Planets to be Injected:

The injected fake planets will be scaled down versions of the unocculted PSF. The following code simply crops and centers this unocculted psf to prepare it for injection

In [ ]:
# read in unocculted PSF
with fits.open("old_simulated_data/NIRCam_unocculted_{0}.fits".format(filtername)) as hdulist:
    psf_cube = hdulist[0].data 
    psf_head = hdulist[0].header
    
# collapse reference psf in time
psf_frame = np.nanmean(psf_cube, axis=0)

# find the centroid
bestfit = fakes.gaussfit2d(psf_frame, 71, 30, searchrad=3, guessfwhm=2, guesspeak=1, refinefit=True)

psf_xcen, psf_ycen = bestfit[2:4]

# recenter PSF to that location
x, y = np.meshgrid(np.arange(-20,20.1,1), np.arange(-20,20.1,1))
x += psf_xcen
y += psf_ycen

psf_stamp = scipy.ndimage.map_coordinates(psf_frame, [y,x])

### Generating a dataset
First, we need to generate data of the science target taken at two telescope roll angles. 

In [ ]:
#Import the dataset to be used
filtername = "f300m"

# read in roll 1
with fits.open("old_simulated_data/NIRCam_target_Roll1_{0}.fits".format(filtername)) as hdulist:
    roll1_cube = hdulist[0].data

# read in roll 2
with fits.open("old_simulated_data/NIRCam_target_Roll2_{0}.fits".format(filtername)) as hdulist:
    roll2_cube = hdulist[0].data  

# combine the two rows
full_seq = np.concatenate([roll1_cube, roll2_cube], axis=0)

# two rolls are offset 10 degrees
pas = np.append([0 for _ in range(roll1_cube.shape[0])], [10 for _ in range(roll2_cube.shape[0])])

# for each image, the (x,y) center where the star is is just the center of the image
centers = np.array([np.array(frame.shape)//2. for frame in full_seq])

# give it some names, just in case we want to refer to them
filenames = np.append(["roll1_{0}".format(i) for i in range(roll1_cube.shape[0])],
                      ["roll2_{0}".format(i) for i in range(roll1_cube.shape[0])])

#Define dataset
dataset = Instrument.GenericData(full_seq, centers, IWA=4, parangs=pas, filenames=filenames)
dataset.flipx = False

### Injecting the fake planets

Now that we have our data and have prepare our fake planet to be injected, we can choose how many we'd like to put into our dataset, what we want their relative fluxes to be, and their separations from the planet. For now, we'll inject three planets into a single dataset.

In [ ]:
#Let's choose our contrasts so that the planets get fainter as we go further from the star
psf_stamp_input = np.array([psf_stamp for j in range(12)])
input_contrasts = [1e-3, 7e-4, 3e-4]
planet_seps = [15, 25, 35]
pas = [0, 90, 180, 270]

#Now injecting the fake planets in a spiral:
for input_contrast, planet_sep in zip(input_contrasts, planet_seps):
    planet_fluxes = psf_stamp_input*input_contrast
    
    for pa in pas:
        fakes.inject_planet(frames = dataset.input, 
                            centers=dataset.centers, 
                            inputflux=planet_fluxes, 
                            astr_hdrs=dataset.wcs, 
                            radius=planet_sep,
                            pa = pa)

plt.imshow(dataset.input[0], interpolation="nearest", cmap="inferno")

### Run KLIP - Recover Planets

Now that we've injected our fake planets, we can check how well we were able to recover them by running KLIP. We'll only use 1 subsection and 1 annulus. 

In [ ]:
 #Set output directory
outputdir = 'contrastcurves'
fileprefix = 'FAKE_KLIP_ADI_A9K5S1M1'
numbasis = [1,5,10,20,50]


#Run KLIP on dataset with injected fakes
parallelized.klip_dataset(dataset, 
                          outputdir=outputdir, 
                          fileprefix=fileprefix, 
                          algo = 'klip', 
                          annuli=1, 
                          subsections=1, 
                          movement=1, 
                          numbasis=numbasis, 
                          mode="ADI")

# Plot this reduced data cube. 
with fits.open("contrastcurves/FAKE_KLIP_ADI_A9K5S1M1-KLmodes-all.fits") as hdulist:
    adi_cube = hdulist[0].data
    
plt.figure()
# plot the KL10 Cube (index of 2)
plt.imshow(adi_cube[2], interpolation='nearest', cmap='inferno')
plt.gca().invert_yaxis()

### Recovering Flux Values

We can now visually inspect how well we were able to recover each injected planet, but if we want to quantify this algrorithm throughput, pyKLIP has a built in function "retrieve_planet_flux" so we can compare the retrieved flux to the input flux

In [ ]:
#Obtain the centers of the output KLIP fits file
with fits.open("contrastcurves/FAKE_KLIP_ADI_A9K5S1M1-KLmodes-all.fits") as hdulist:
    cube = hdulist[0].data[1]
    cube_centers = [hdulist[0].header['PSFCENTX'], hdulist[0].header['PSFCENTY']]

#Create and empty list to store retrieved flux values
retrieved_fluxes = []

#Retrieve planet fluxes
for input_contrast, planet_sep in zip(input_contrasts, planet_seps):
    
    fake_planet_fluxes = []
                                      
    for pa in pas:
        fake_flux = fakes.retrieve_planet_flux(frames = cube, 
                                            centers = cube_centers,
                                            astr_hdrs = dataset.output_wcs[0], 
                                            sep = planet_sep,
                                            pa = pa)
        fake_planet_fluxes.append(fake_flux)

    retrieved_fluxes.append(fake_flux)

### Calculating throughput

Now that we've run a full reduction on the data with injected planets, we can calculate the throughput to figure our how well we were able to recover the planet at different separations. 

Throughput can be calculated as follows: throughput = $\frac{output\ flux}{input\ flux}$. We'll need to calculate the input flux first by multiplying our input contrasts by the peak star flux estimated from our previous Gaussian fit.

In [ ]:
#Calculate the input flux
input_flux = [contrast*bestfit[0] for contrast in input_contrasts]

#Put everything in table form (as a dataframe)
throughput_table = pd.DataFrame(
    {'retrieved_fluxes': retrieved_fluxes,
     'input_flux': input_flux,
     'separation': planet_seps
     })

#Calculate througput and put it in a column in the dataframe

throughput_table["throughput"] = throughput_table["retrieved_fluxes"]/throughput_table["input_flux"]

### Now we can visualize our throughput as a function of separation

In [ ]:
plt.plot(throughput_table["separation"], throughput_table["throughput"])
plt.xlabel("Separation (pixels)")
plt.ylabel("Throughput")
plt.title("Throughput as a Function of Separation from Star")

## Improving throughput

From this plot, we can see that our throughput improves as a function of separation from the host star. Still, our plot is could benefit from a few more data points. If we could inject fake planets in every available space in our data, we'd get a much better sense of how throughput varies as a function of separation. However, this turns out to be quite tedious and messy. Instead, we'll inject only a few fake planets in an image, but do this for different separations in multiple images and record the throughput for each image. This will give us the same information in a much cleaner way.   

To do this, we'll create a loop that injects 10 planets at a set list of separations, but changing the position angle each time. Every time it injects a new set of fake planets, it runs pyKLIP on the injected dataset, and then retrieves the fluxes of the fake planets from the KLIP-ed data to be output into a table.  
Since we're using a loop to inject fake planets in new locations each time, we also need to regenerate a new dataset each time. Here we'll generate 10 datasets.

In [ ]:
datasets = []
for dataset in range(10):
    
    #Import the dataset to be used
    filtername = "f300m"

    # read in roll 1
    with fits.open("old_simulated_data/NIRCam_target_Roll1_{0}.fits".format(filtername)) as hdulist:
        roll1_cube = hdulist[0].data

    # read in roll 2
    with fits.open("old_simulated_data/NIRCam_target_Roll2_{0}.fits".format(filtername)) as hdulist:
        roll2_cube = hdulist[0].data  

    # combine the two rows
    full_seq = np.concatenate([roll1_cube, roll2_cube], axis=0)

    # two rolls are offset 10 degrees
    pas = np.append([0 for _ in range(roll1_cube.shape[0])], [10 for _ in range(roll2_cube.shape[0])])

    # for each image, the (x,y) center where the star is is just the center of the image
    centers = np.array([np.array(frame.shape)//2. for frame in full_seq])

    # give it some names, just in case we want to refer to them
    filenames = np.append(["roll1_{0}".format(i) for i in range(roll1_cube.shape[0])],
                          ["roll2_{0}".format(i) for i in range(roll1_cube.shape[0])])

    #Define dataset
    dataset = Instrument.GenericData(full_seq, centers, IWA=4, parangs=pas, filenames=filenames)
    dataset.flipx = False
    datasets.append(dataset)

### Begin the loop:

1) Use our generated list of ten datasets  
2) Specify input contrasts and planet separations  
3) Specify starting position angle and amount by which to change position angle each loop
4) Inject fake planets with  fakes.inject_planet  
5) Run KLIP on fake injected dataset  
6) Read in the KLIP-ed dataset  
7) Retrieved fluxes of injected planets with fakes.retrieve_planet_flux  
8) Append retrieved fluxes, planet separtions and contrasts to a list  

In [ ]:
def test_multiple_planets(input_contrasts, planet_seps, pas, pas_step_size):
    
    pas_all = [] 
    retrieved_fluxes_all = []
    planet_seps_all = [] 
    input_contrasts_all = []
    
    #started changing pas
    for dataset_num, dataset in enumerate(datasets):
        
        psf_stamp_input = np.array([psf_stamp for j in range(12)])
        pas_array = [(x+pas_step_size*dataset_num*np.ones(12))-dataset.PAs for x in pas]
        
        start_over = False
        for input_contrast, planet_sep, pa in zip(input_contrasts, planet_seps, pas):
         
            check_sep_x = planet_sep*np.cos((pa + 90)) 
            check_sep_y = planet_sep*np.sin((pa + 90))
            
            dist_p1 = np.sqrt((check_sep_x - planet1_peak_x)**2 + (check_sep_y - planet1_peak_y)**2)
            dist_p2 = np.sqrt((check_sep_x - planet2_peak_x)**2 + (check_sep_y - planet2_peak_y)**2)
            
            if dist_p1 > 12 and dist_p2 > 12:
                planet_fluxes = psf_stamp_input*input_contrast


                fakes.inject_planet(frames = dataset.input, 
                                    centers=dataset.centers, 
                                    inputflux=planet_fluxes, 
                                    astr_hdrs=dataset.wcs, 
                                    radius=planet_sep,
                                    pa = pa)
            else:
                start_over = True
                
        if start_over:
            continue
            


        #Set output directory
        outputdir = 'contrastcurves'
        fileprefix = 'FAKE_KLIP_ADI_A9K5S4M1' + str(dataset_num)
        numbasis = [1,5,10,20,50]


        #Run KLIP on dataset with injected fakes
        parallelized.klip_dataset(dataset, 
                                  outputdir=outputdir, 
                                  fileprefix=fileprefix, 
                                  algo = 'klip', 
                                  annuli=1, 
                                  subsections=1, 
                                  movement=1, 
                                  numbasis=numbasis, 
                                  mode="ADI")


        # replace os.path.join(foldername,filename)
        klipdataset = "contrastcurves/"+ fileprefix + "-KLmodes-all.fits"
        with fits.open(klipdataset) as hdulist:
            outputfile = hdulist[0].data
            outputfile_centers = [hdulist[0].header['PSFCENTX'], hdulist[0].header['PSFCENTY']]

        outputfile_frame = outputfile[2]

        thetas_retrieve = [x+thetas_step_size*dataset_num for x in thetas]
        retrieved_planet_fluxes = []


        #retrieving planet flux
        for input_contrast, planet_sep, pa in zip(input_contrasts, planet_seps, pas):

            fake_flux = fakes.retrieve_planet_flux(frames = outputfile_frame, 
                                                    centers=outputfile_centers,
                                                    astr_hdrs=dataset.output_wcs[0], 
                                                    sep=planet_sep,
                                                    pa = pa,
                                                    searchrad = 7)

            retrieved_planet_fluxes.append(fake_flux)
            
        
        retrieved_fluxes_all.append(retrieved_planet_fluxes)
        pas_all.append(pas)
        planet_seps_all.append(planet_seps)
        input_contrasts_all.append[input_contrasts]
        
        
    return pas_all, retrieved_fluxes_all, planet_seps_all, input_contrasts_all
          

In [ ]:
pas, retrieved_fluxes, planet_seps, input_contrasts = test_multiple_planets(input_contrasts = [3e-3, 2e-3, 1e-3, 3e-3],
          planet_seps = [10,12, 15,20],
          pas = [0, 60, 210, 270], 
          pas_step_size = 75)

### Calculate throughput

We can first create a table using the lists of retrieved planets fluxes, their separations and input fluxes. Then, we'll add the column "throughput" which will hold the throughput calculations for each value in the table. The throughput calculation is: $\frac{Retrieved\ flux}{Input\ flux}$

In [ ]:
tables = []
#Loop through each list to create a table of all variables
for flux, sep, input_contrast, pa in zip(retrieved_fluxes_all, planet_seps_all, input_contrasts_all, pas_all):
    input_flux = np.array(input_contrast)*bestfit[0]
    t = Table([flux,sep, input_flux, pas], names = ('flux', 'separation', 'input_flux', 'thetas'))
    
    tables.append(t)
flux_sep = vstack([x for x in tables])
#Calculate throughput and add it to list 
flux_sep['throughput'] = flux_sep['flux']/flux_sep['input_flux']

#We can also calculate the median througput per separation

#Group by separation
flux_by_sep = flux_sep.group_by('separation')

#Calculate the median value for each separation group
med_flux_by_sep = flux_by_sep.groups.aggregate(np.median)

In [ ]:
plt.figure(figsize = (15,10))
plt.scatter(flux_sep["separation"], flux_sep["throughput"], color = '#BC96E6', label = "Calculated Throughputs")
plt.plot(med_flux_by_sep["separation"], med_flux_by_sep["throughput"], color = 'red', label = "Median Throughput")
plt.ylabel("Throughput")
plt.xlabel("Planet Separation")
plt.title("Algorithm Throughput")
plt.legend(frameon = False)
plt.rcParams.update({'font.size': 22})
plt.savefig("throughput_med.png")